In [39]:
import numpy as np
import pandas as pd
import feather

from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
  
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
  
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import (LinearRegression, Lasso, ElasticNet, Ridge,
                                  Perceptron)
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB

from sklearn.utils.testing import assert_array_equal
from sklearn.utils.testing import assert_equal
from sklearn.utils.testing import assert_almost_equal
from sklearn.utils.testing import assert_true
from sklearn.utils.testing import assert_false
from sklearn.utils.testing import assert_raises
  
from sklearn.utils.testing import assert_greater
import pickle

from sklearn.metrics import log_loss, confusion_matrix, f1_score, accuracy_score

In [3]:
df_train = np.load('../cache/train_stage2_fe2.npy')

In [5]:
df_test = np.load('../cache/test_stage2_fe2.npy')

In [7]:
df = pd.read_csv('../cache/train_stage2_labels.csv')

In [8]:
df.head()

,y
0,1
1,2
2,2
3,3
4,4


In [10]:
y = df['y'].values

In [11]:
x1 = np.load('../cache/train_stage2_x1.npy')
x2 = np.load('../cache/train_stage2_x2.npy')
y1 = np.load('../cache/train_stage2_y1.npy')
y2 = np.load('../cache/train_stage2_y2.npy')

wts_per_class = np.load('../cache/stage2_train_weights_per_class.npy')
wts_per_class = wts_per_class.tolist()
print(wts_per_class)

w1 = np.array([wts_per_class[j] for j in y1], )
w2 = np.array([wts_per_class[j] for j in y2], )

w = np.array([wts_per_class[j] for j in y], )

{1: 4.572507552870091, 2: 6.407630522088353, 3: 37.427083333333336, 4: 3.912117177097204, 5: 12.816479400749063, 6: 11.42087542087542, 7: 2.5, 8: 174.66666666666666, 9: 84.79069767441861}


In [28]:
ovo = OneVsOneClassifier(LinearSVC(random_state=0))
Cs = [0.1, 0.01, 0.03, 0.05, 0.08, 0.5, 0.8]
cv = GridSearchCV(ovo, {'estimator__C': Cs}, verbose=2, refit=True)
cv.fit(df_train, y)
best_C = cv.best_estimator_.estimators_[0].C
assert_true(best_C in Cs)


Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] estimator__C=0.1 ................................................
[CV] ....................................... estimator__C=0.1 -  13.5s
[CV] estimator__C=0.1 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.5s remaining:    0.0s


[CV] ....................................... estimator__C=0.1 -  13.3s
[CV] estimator__C=0.1 ................................................
[CV] ....................................... estimator__C=0.1 -  13.1s
[CV] estimator__C=0.01 ...............................................
[CV] ...................................... estimator__C=0.01 -  13.2s
[CV] estimator__C=0.01 ...............................................
[CV] ...................................... estimator__C=0.01 -  13.4s
[CV] estimator__C=0.01 ...............................................
[CV] ...................................... estimator__C=0.01 -  13.0s
[CV] estimator__C=0.03 ...............................................
[CV] ...................................... estimator__C=0.03 -  13.2s
[CV] estimator__C=0.03 ...............................................
[CV] ...................................... estimator__C=0.03 -  13.3s
[CV] estimator__C=0.03 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.3min finished


In [29]:
print(best_C)

0.1


In [30]:
test_preds = cv.predict(df_test)
print(test_preds)

[1 1 4 4 1 1 1 4 1 7 1 7 7 7 1 1 1 7 1 1 1 1 7 1 1 1 4 2 4 1 1 7 1 1 1 2 1
 7 1 1 1 1 1 4 7 2 1 1 7 7 1 7 2 1 4 1 4 1 1 1 1 1 1 2 1 4 1 1 7 1 4 1 1 7
 1 4 7 1 1 1 1 1 1 7 1 1 1 1 1 1 1 1 1 1 7 1 1 1 1 1 1 7 1 1 4 1 1 1 1 1 7
 4 1 7 7 1 4 1 1 1 1 7 1 1 1 1 1 1 7 7 1 4 1 7 1 1 1 7 4 1 1 1 1 1 1 7 1 1
 1 2 4 1 4 1 1 7 1 1 1 2 1 4 1 7 1 1 2 4 4 1 4 1 7 1 7 1 1 4 1 1 1 1 1 1 4
 4 1 1 1 1 7 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 7 2 1 7 7 7 1 7 7 1 1 1
 1 1 7 1 1 1 4 1 1 1 4 1 1 1 1 7 1 4 1 1 1 1 1 1 7 1 7 7 7 4 1 1 7 1 1 1 1
 4 7 4 1 1 7 1 4 1 1 7 2 1 7 1 1 7 1 7 7 1 1 1 1 1 1 7 1 7 1 1 1 4 7 1 1 7
 1 1 1 1 4 7 1 7 1 7 1 1 1 1 4 4 1 1 1 1 2 7 7 1 4 1 1 1 7 1 7 7 1 1 4 1 1
 1 1 7 2 1 1 1 4 1 1 1 4 1 4 1 1 4 7 1 4 2 1 7 1 1 1 4 1 7 7 2 1 4 7 7 2 1
 1 7 1 7 4 7 1 7 1 1 1 1 1 1 1 1 7 1 1 1 1 1 1 1 1 1 1 1 1 1 1 7 7 1 2 1 1
 1 4 1 1 1 4 1 1 7 4 1 1 7 1 1 1 7 7 1 1 1 1 1 1 1 1 7 7 1 7 1 1 4 7 7 1 1
 1 4 1 1 4 7 1 1 1 1 1 7 1 1 1 1 1 1 1 7 7 2 1 1 7 7 1 7 1 1 1 1 7 1 1 1 4
 4 7 1 1 1 7 4 7 1 1 7 1 

In [20]:
df = pd.DataFrame()
df['Class1'] =0
df['Class2'] =0
df['Class3'] =0
df['Class4'] =0
df['Class5'] =0
df['Class6'] =0
df['Class7'] =0
df['Class8'] =0
df['Class9'] =0

In [21]:
df['Class1'] = [0.95 if i == 0 else 0.05 for i in test_preds]
df['Class2'] = [0.95 if i == 1 else 0.05 for i in test_preds]
df['Class3'] = [0.95 if i == 2 else 0.05 for i in test_preds]
df['Class4'] = [0.95 if i == 3 else 0.05 for i in test_preds]
df['Class5'] = [0.95 if i == 4 else 0.05 for i in test_preds]
df['Class6'] = [0.95 if i == 5 else 0.05 for i in test_preds]
df['Class7'] = [0.95 if i == 6 else 0.05 for i in test_preds]
df['Class8'] = [0.95 if i == 7 else 0.05 for i in test_preds]
df['Class9'] = [0.95 if i == 8 else 0.05 for i in test_preds]

In [22]:
df.head()

,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9
0,0.05,0.95,0.05,0.05,0.05,0.05,0.05,0.05,0.05
1,0.05,0.95,0.05,0.05,0.05,0.05,0.05,0.05,0.05
2,0.05,0.05,0.05,0.05,0.95,0.05,0.05,0.05,0.05
3,0.05,0.05,0.05,0.05,0.95,0.05,0.05,0.05,0.05
4,0.05,0.95,0.05,0.05,0.05,0.05,0.05,0.05,0.05


In [23]:
df1 = pd.read_csv('../cache/test_id.csv')

In [24]:
df1.head()

,ID
0,1
1,2
2,3
3,4
4,5


In [25]:
df['ID'] = df1['ID'].values

In [26]:
df.to_csv('../submissions/sub_svc_one_on_one.csv', index=None)
# 2.25939 on stage2 private LB, 2.90524 on stage2 public LB

In [31]:
clf = OneVsOneClassifier(LinearSVC(random_state=0, C=0.1))

In [34]:
clf.fit(x1,y1)

OneVsOneClassifier(estimator=LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [37]:
test_preds = clf.predict(x2)
print(test_preds)

[4 4 4 4 7 4 4 4 4 4 4 4 4 1 4 1 7 1 1 4 7 4 4 4 4 7 7 4 4 6 4 7 4 4 4 4 1
 4 1 4 4 4 4 7 4 4 1 4 4 4 4 1 4 1 1 4 4 4 4 1 6 1 4 4 4 4 4 6 1 4 4 7 4 4
 4 4 4 4 4 4 7 4 4 4 7 4 7 7 4 4 4 4 7 4 4 4 4 4 4 4 4 4 4 4 1 4 7 4 4 7 4
 4 1 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 1 4 4 4 4 4 4 4 4 1 4 1 4 4 4 4 4 4 1 4
 4 4 4 7 4 4 1 4 4 4 4 4 1 4 4 4 4 4 1 4 1 4 4 7 4 4 4 1 4 1 4 6 7 4 4 4 1
 4 7 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 6 4 4 4 4 4 4 4 4 4 4 4 4 1 4 7 4
 4 4 4 4 7 4 4 4 4 4 1 1 1 4 4 4 4 4 4 1 4 4 4 4 4 6 4 4 4 4 4 4 1 1 4 4 7
 4 4 4 4 1 6 4 1 6 4 6 4 1 4 4 4 4 7 4 4 4 4 4 4 1 4 4 4 4 4 4 2 1 4 4 4 7
 4 4 4 1 4 4 4 4 7 4 4 4 4 4 7 4 4 7 4 4 4 4 1 4 4 4 4 4 1 7 1 1 4 4 4 4 4
 1 1 4 4 7 4 1 7 7 4 4 7 7 4 7 4 4 4 1 4 4 4 4 4 1 4 1 4 4 4 4 4 4 1 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 6 2 4 1 7 4 4 4 7 4 4 4 1 1 7 7 4 4 4 7 4 6
 7 4 4 7 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 1 1 4 1 4 7 1 4 4 4 1 4 6
 7 4 4 4 4 2 4 4 4 2 4 1 4 7 7 4 4 4 4 4 7 1 4 4 7 4 4 7 4 4 4 1 4 4 4 4 4
 4 4 4 1 4 1 4 7 7 4 1 4 

In [42]:
y2

array([2, 2, 2, 7, 7, 7, 4, 7, 3, 4, 7, 1, 3, 1, 7, 6, 5, 7, 4, 5, 5, 4, 4,
       4, 2, 6, 7, 7, 6, 2, 1, 1, 1, 4, 6, 7, 7, 1, 7, 5, 1, 2, 7, 3, 7, 7,
       6, 4, 7, 7, 2, 7, 5, 7, 7, 4, 7, 4, 5, 4, 4, 4, 4, 2, 6, 5, 7, 7, 3,
       4, 7, 7, 4, 4, 4, 4, 7, 7, 6, 4, 4, 6, 4, 7, 5, 7, 2, 2, 3, 2, 7, 7,
       2, 1, 1, 7, 7, 1, 6, 7, 2, 7, 4, 4, 9, 4, 5, 7, 1, 4, 6, 4, 1, 9, 4,
       5, 4, 4, 1, 1, 5, 6, 2, 1, 2, 5, 7, 7, 1, 7, 1, 5, 7, 7, 6, 2, 4, 4,
       5, 6, 7, 7, 3, 2, 3, 4, 6, 5, 7, 7, 1, 4, 4, 4, 7, 1, 5, 7, 9, 2, 4,
       5, 7, 4, 1, 6, 4, 5, 1, 4, 7, 7, 7, 4, 6, 7, 7, 7, 8, 6, 7, 4, 5, 7,
       7, 1, 1, 7, 2, 7, 4, 4, 7, 7, 6, 1, 6, 5, 4, 5, 7, 4, 6, 4, 2, 7, 8,
       1, 2, 3, 9, 2, 7, 5, 2, 4, 2, 2, 7, 2, 4, 1, 2, 6, 2, 6, 2, 2, 7, 7,
       4, 4, 1, 1, 1, 1, 7, 3, 1, 2, 7, 2, 4, 7, 1, 6, 2, 7, 4, 1, 4, 5, 1,
       1, 7, 2, 7, 1, 7, 7, 7, 2, 1, 7, 7, 2, 7, 7, 7, 2, 6, 2, 7, 7, 1, 7,
       1, 7, 7, 6, 1, 4, 1, 7, 4, 7, 4, 1, 7, 1, 1, 7, 4, 1, 1, 2, 4, 1, 4,
       2, 5,

In [58]:
from utils import *
score2 = log_loss(y2, test_preds)
# score2 = multi_log_loss(y2, test_preds, labels = list(range(1,10)))
print('CV multi_log_loss: {}'.format(score2))

fscore = f1_score(y2, test_preds, average='macro')
print('CV f1_score: {}'.format(fscore))

acc = accuracy_score(y2, test_preds)
print('CV accuracy: {}'.format(acc))

print(confusion_matrix(y2, test_preds, labels = list(range(1,10))))

CV multi_log_loss: -114.94471360052813
CV f1_score: 0.07517463719109137
CV accuracy: 0.21002710027100271
[[ 18   4   0  99   0   0   8   0   0]
 [ 11   0   0  78   0   2  15   0   0]
 [  3   0   0  18   0   0   3   0   0]
 [ 13   2   1 113   0   1  11   0   0]
 [  6   0   0  39   0   0   9   0   0]
 [  9   1   0  42   0   1   2   0   0]
 [ 34   5   0 149   0  10  23   0   0]
 [  0   0   0   2   0   0   0   0   0]
 [  1   0   0   3   0   0   2   0   0]]


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [59]:
clf = OneVsRestClassifier(svm.SVC()).fit(df_train, y)
Y_pred = clf.predict(df_test)


In [60]:
save_classifier('../cache/svm_svr_clf.pkl', clf)

In [61]:
print(Y_pred)

[4 1 7 7 7 7 7 7 7 7 7 2 1 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 4 7 7 7 1 7 7 7 6
 7 7 7 7 7 7 1 4 7 7 7 4 7 7 1 7 7 7 7 4 7 7 6 7 7 7 1 6 7 7 8 7 7 7 1 7 7
 3 7 1 7 5 7 7 5 7 7 7 7 7 1 7 7 7 5 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 5 7
 6 7 1 7 7 7 1 7 7 6 7 2 7 7 7 1 7 7 7 7 7 7 7 7 7 7 7 4 7 7 7 7 7 7 7 4 7
 7 7 7 5 7 1 7 7 7 7 7 7 7 7 7 1 7 7 7 7 1 7 7 7 1 1 7 7 7 4 7 7 7 7 7 7 7
 4 7 7 7 7 7 7 4 5 7 4 1 6 7 7 7 1 7 7 1 7 7 7 7 7 7 4 7 7 7 5 2 7 7 7 7 7
 4 7 7 7 7 7 7 7 7 7 7 7 1 1 1 7 4 1 7 7 7 7 7 7 7 1 7 7 7 7 7 7 7 1 7 7 4
 7 7 7 7 5 1 7 7 4 7 7 6 7 7 7 7 7 1 7 7 7 7 7 4 7 7 1 7 7 7 7 5 4 7 7 2 7
 1 2 7 7 1 7 7 7 4 7 1 1 7 7 4 4 7 7 6 7 4 7 4 7 7 7 7 7 7 7 7 1 2 7 7 4 1
 7 7 7 7 4 7 7 7 7 2 7 7 2 5 7 7 7 7 7 4 7 7 7 6 1 1 7 7 7 7 9 7 7 7 4 1 7
 2 1 1 7 7 1 7 1 7 7 7 7 7 7 5 7 7 7 7 4 7 7 7 7 7 7 7 6 5 7 7 7 7 4 7 7 7
 7 2 7 7 7 1 6 2 5 7 7 7 6 7 7 7 7 7 7 7 1 5 4 7 7 7 7 7 7 7 1 7 7 7 7 7 7
 5 7 7 7 7 1 7 7 7 7 1 4 7 7 7 1 1 7 7 7 7 7 1 7 7 7 7 7 7 1 7 7 7 6 4 7 7
 2 7 6 7 7 7 7 6 5 7 7 7 

In [62]:
df = pd.DataFrame()
df['Class1'] =0
df['Class2'] =0
df['Class3'] =0
df['Class4'] =0
df['Class5'] =0
df['Class6'] =0
df['Class7'] =0
df['Class8'] =0
df['Class9'] =0

df['Class1'] = [0.95 if i == 0 else 0.05 for i in Y_pred]
df['Class2'] = [0.95 if i == 1 else 0.05 for i in Y_pred]
df['Class3'] = [0.95 if i == 2 else 0.05 for i in Y_pred]
df['Class4'] = [0.95 if i == 3 else 0.05 for i in Y_pred]
df['Class5'] = [0.95 if i == 4 else 0.05 for i in Y_pred]
df['Class6'] = [0.95 if i == 5 else 0.05 for i in Y_pred]
df['Class7'] = [0.95 if i == 6 else 0.05 for i in Y_pred]
df['Class8'] = [0.95 if i == 7 else 0.05 for i in Y_pred]
df['Class9'] = [0.95 if i == 8 else 0.05 for i in Y_pred]

In [63]:
df['ID'] = df1['ID'].values

In [64]:
df.to_csv('../submissions/sub_svm_one_vs_rest.csv', index=None)

In [ ]:
# 2.54206 on stage2 private leaderboard, 3.16635 on stage2 public leaderboard

In [65]:
clf.fit(x1,y1)
test_preds = clf.predict(x2)

score2 = log_loss(y2, test_preds)
# score2 = multi_log_loss(y2, test_preds, labels = list(range(1,10)))
print('CV multi_log_loss: {}'.format(score2))

fscore = f1_score(y2, test_preds, average='macro')
print('CV f1_score: {}'.format(fscore))

acc = accuracy_score(y2, test_preds)
print('CV accuracy: {}'.format(acc))

print(confusion_matrix(y2, test_preds, labels = list(range(1,10))))

CV multi_log_loss: -114.94471360052813
CV f1_score: 0.05121075194067895
CV accuracy: 0.2994579945799458
[[  0   0   0   0   0   0 129   0   0]
 [  0   0   0   0   0   0 106   0   0]
 [  0   0   0   0   0   0  24   0   0]
 [  0   0   0   0   0   0 141   0   0]
 [  0   0   0   0   0   0  54   0   0]
 [  0   0   0   0   0   0  55   0   0]
 [  0   0   0   0   0   0 221   0   0]
 [  0   0   0   0   0   0   2   0   0]
 [  0   0   0   0   0   0   6   0   0]]


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [67]:
ovo = OneVsOneClassifier(MultinomialNB())
Y_pred =ovo.fit(abs(df_train), y).predict(df_test)

In [68]:
df = pd.DataFrame()
df['Class1'] =0
df['Class2'] =0
df['Class3'] =0
df['Class4'] =0
df['Class5'] =0
df['Class6'] =0
df['Class7'] =0
df['Class8'] =0
df['Class9'] =0

df['Class1'] = [0.95 if i == 0 else 0.05 for i in Y_pred]
df['Class2'] = [0.95 if i == 1 else 0.05 for i in Y_pred]
df['Class3'] = [0.95 if i == 2 else 0.05 for i in Y_pred]
df['Class4'] = [0.95 if i == 3 else 0.05 for i in Y_pred]
df['Class5'] = [0.95 if i == 4 else 0.05 for i in Y_pred]
df['Class6'] = [0.95 if i == 5 else 0.05 for i in Y_pred]
df['Class7'] = [0.95 if i == 6 else 0.05 for i in Y_pred]
df['Class8'] = [0.95 if i == 7 else 0.05 for i in Y_pred]
df['Class9'] = [0.95 if i == 8 else 0.05 for i in Y_pred]
df['ID'] = df1['ID'].values
df.to_csv('../submissions/sub_nb_one_on_one.csv', index=None)

In [ ]:
# 2.89574 on stage2 private leaderboard, 2.95424 on stage2 public leaderboard

In [75]:
ovo.fit(abs(x1),y1)
test_preds = ovo.predict(x2)

score2 = log_loss(y2, test_preds)
# score2 = multi_log_loss(y2, test_preds, labels = list(range(1,10)))
print('CV multi_log_loss: {}'.format(score2))

fscore = f1_score(y2, test_preds, average='weighted')
print('CV f1_score: {}'.format(fscore))

acc = accuracy_score(y2, test_preds)
print('CV accuracy: {}'.format(acc))

print(confusion_matrix(y2, test_preds, labels = list(range(1,10))))

CV multi_log_loss: -114.94471360052813
CV f1_score: 0.13107980028068605
CV accuracy: 0.10298102981029811
[[ 6  2 41  6 12  8  5 45  4]
 [ 1 13 35  2 13  4  9 28  1]
 [ 1  1  8  1  5  2  0  5  1]
 [ 2  1 73  6  6  1  3 43  6]
 [ 2  0 10  1 17  3  5 15  1]
 [ 1  2 17  1 18  4  0 10  2]
 [ 3 17 62  3 16  7 22 88  3]
 [ 0  1  0  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0  5  0]]
